`Дисциплина: Методы и технологии машинного обучения`   
`Уровень подготовки: бакалавриат`   
`Направление подготовки: 01.03.02 Прикладная математика и информатика`   
`Семестр: осень 2021/2022`   




# Лабораторная работа №2: Параметрические классификаторы: логистическая регрессия, LDA, QDA 

В практических примерах ниже показано:   

* как импортировать данные из .csv
* как рассчитать матрицу неточностей
* как считать показатели качества модели по матрице неточностей (метод проверочной выборки)
* как пользоваться моделями логистической регрессии, линейного и квадратичного дискриминантного анализа  

*Модели*: логистическая регрессия, LDA, QDA.   
*Данные*: `Default`.   


# Указания к выполнению


## Загружаем пакеты

In [ ]:
# загрузка пакетов: инструменты
import numpy as np                                                      # работа с массивами
import pandas as pd                                                     # фреймы данных
import random                                                           # генератор случайных чисел
import matplotlib as mpl                                                # графики
import seaborn as sns                                                   # стили и шаблоны графиков на основе matplotlib
from scipy.stats import shapiro                                         # тест Шапиро-Уилка на нормальность распределения
from statsmodels.stats.diagnostic import lilliefors                     # тест Лиллиефорса на нормальность распределения

# загрузка пакетов: модели
from sklearn.linear_model import LogisticRegression                     # логистическая регрессия (ММП)
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis    # LDA
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis # QDA
from sklearn.metrics import classification_report, confusion_matrix     # матрица неточностей
from sklearn.metrics import plot_confusion_matrix                       # визуализация матрицы неточностей
from sklearn.metrics import precision_score                             # PPV (TP / (TP + FP))
from sklearn.metrics import precision_recall_fscore_support             # расчёт TPR, SPC, F1
from sklearn.metrics import plot_roc_curve, roc_curve, auc              # ROC-кривая
from statsmodels.api import add_constant                                # подготовка матрицы X для модели регрессии
from statsmodels.formula.api import logit                               # модель логистической регрессии

# встроить графики в блокнот 
%matplotlib inline 
# настройка стиля и отображения графиков
#  примеры стилей и шаблонов графиков: http://tonysyu.github.io/raw_content/matplotlib-style-gallery/gallery.html
mpl.style.use('seaborn-whitegrid')
#  устанавливаем палитру для графиков
# sns.color_palette("Set2") # раскомментируйте, чтобы посмотреть палитру
sns.set_palette("Set2")

In [ ]:
# константы
#  ядро для генератора случайных чисел
my_seed = 9212
plt = mpl.pyplot

## Загружаем данные

Набор данных `Default` в формате .csv доступен по адресу: <https://raw.githubusercontent.com/aksyuk/R-data/master/ISLR/Default.csv>.  

In [ ]:
# читаем таблицу из файла .csv во фрейм
DF = 

# делаем из категориальных (в нашем случае бинарных) переменных факторы, т.е. перенумеровываем уникальные значения



# первые 5 строк фрейма
DF.head(5)

In [ ]:
# типы столбцов фрейма
DF.dtypes

In [ ]:
# сколько наблюдений во фрейме
print("Число наблюдений во фрейме DF:\n", len(DF.index))

---

📚 **Подробнее о функции `factorize()`**

In [ ]:
# способ 1: кодируем категории по встречаемости в данных
#  кодируем массив, который начинается с Yes, по умолчанию
labels1, uniques1 = pd.factorize(['Yes', 'No', 'Yes', 'Yes', 'No', 'No'])

print("Пронумеровенные значения: \n", labels1)
print("Уникальные коды: \n", uniques1)

In [ ]:
# способ 2: сортируем категории по алфавиту, потом кодируем
#  кодируем массив, который начинается с Yes, с сортировкой кодов (уникальных значений) по алфавиту
labels2, uniques2 = pd.factorize(['Yes', 'No', 'Yes', 'Yes', 'No', 'No'], sort=True)

print("Пронумеровенные значения: \n", labels2)
print("Уникальные коды: \n", uniques2)

In [ ]:
# способ 3: полный контроль
#  сначала создаём словарь
x_dict = {'Yes' : 1,
         'No' : 0}
# теперь определяем x_to_factorize как столбец фрейма df_temp
df_tmp = pd.DataFrame({'x_to_factorize': ['Yes', 'No', 'Yes', 'Yes', 'No', 'No']})
# создаём столбец с кодами под названием x_factor с помощью map()
df_tmp['x_factor'] = df_tmp.x_to_factorize.map(x_dict)

df_tmp

---

## Предварительный анализ данных   

В этой лабораторной для оценки точности моделей мы используем метод проверочной выборки. Создадим фреймы с обучающей и тестовой выборками (`DF_train` и `DF_test` соответственно), распределив наблюдения случайным образом в соотношении 80% и 20%.   

In [ ]:
# обучающая выборка
DF_train = 
# тестовая выборка (методом исключения)
DF_test = 

# сколько наблюдений в обучающей выборке + подсчёт частот классов
print("Число наблюдений во фрейме DF_train:\n", ,
     "\n\nЧастоты классов defaultYes:\n",
     , sep='')

In [ ]:
# сколько наблюдений в тестовой выборке + подсчёт частот классов
print("Число наблюдений во фрейме DF_test:\n", ,
     "\n\nЧастоты классов defaultYes:\n",
     , sep='')

Посмотрим на разброс значений переменных и взаимосвязи между ними в обучающей выборке.   

In [ ]:
# создаём полотно и делим его на три части
fig = plt.figure(figsize=(12,5))
gs = mpl.gridspec.GridSpec(1, 4)
ax1 = plt.subplot(gs[0,:-2])
ax2 = plt.subplot(gs[0,-2])
ax3 = plt.subplot(gs[0,-1])

# график разброса
#  класс Yes
ax1.scatter(, 
            , 
            marker='+', linewidths=1, alpha=.8, s=60)
#  класс No
ax1.scatter(, 
            , marker='.', linewidths=1, alpha=.2, s=60)
# подписи осей для графика разброса
ax1.set_xlabel('Balance')
ax1.set_ylabel('Income')

# строим коробчатые
sns.boxplot(x='default', y='balance', data=DF_train, orient='v', ax=ax2)
sns.boxplot(x='default', y='income', data=DF_train, orient='v', ax=ax3)

# корректируем расположение графиков на полотне
gs.tight_layout(plt.gcf())

## Логистическая регрессия  

Классифицируем наблюдения по классам из `defaultYes` с помощью логистической регрессии. В качестве объясняющей переменной возьмём `balance`.  

### Строим модель с помощью пакета `scikit-learn`   

Воспользуемся функцией `LogisticRegression()`.   

In [ ]:
# данные для логистической выборки в формате, понятном функции LogisticRegression()
X_train = 
y_train = 

# строим модель на обучающей
fit_LR_1 = 

# коэффициенты модели
print('Коэффициенты при объясняющих переменных:', np.around(fit_LR_1.coef_, 4),
      '\nКонстанта:', np.around(fit_LR_1.intercept_, 4))

### Строим модель с помощью пакета `statmodels`   

Воспользуемся функцией `logit()`.   

In [ ]:
# строим модель на обучающей
fit_logit_1 = 
# отчёт по коэффициентам


### График фактических и модельных вероятностей      

In [ ]:
# данные тестовой выборки 
X_test = 
y_test = 

# равноотстоящие координаты balance по возрастанию (для графика модельных значений)
x_line_train = np.linspace(DF_train.balance.min(), DF_train.balance.max(), 2000).reshape(-1, 1)
x_line_test = np.linspace(DF_test.balance.min(), DF_test.balance.max(), 2000).reshape(-1, 1)

# прогноз вероятностей для обучающей
y_line_train = 

In [ ]:
# смотрим, как выглядят прогнозы
pd.DataFrame(y_line_train)

In [ ]:
# прогноз вероятностей для тестовой
y_line_test = 

# график логистической регрессии
fig, (ax1, ax2) = plt.subplots(1,2, figsize=(12,5))

# палитра
clrs = sns.color_palette('Set2')

# график для обучающей выборки
#  фактические наблюдения
ax1.scatter(, , 
            marker='|', color=clrs[0], alpha=0.6, label='факт')
#  модельная кривая
ax1.plot(, , 
         linestyle='solid', color=clrs[1], label='модель')
#  заголовок
ax1.set_title('Обучающая выборка')

# график для тестовой выборки
#  фактические наблюдения
ax2.scatter(X_test, y_test, 
            marker='|', color=clrs[0], lw=2, alpha=0.6, label='факт')
#  модельная кривая
ax2.plot(x_line_test, y_line_test[:, 1],
         linestyle='solid', lw=2, color=clrs[1], label='модель')
#  заголовок
ax2.set_title('Тестовая выборка')

# дополнительные настройки графиков
for ax in fig.axes:
    #  линиии вероятностей P=0 и P=1
    ax.hlines(1, xmin=ax.xaxis.get_data_interval()[0],
              xmax=ax.xaxis.get_data_interval()[1], linestyles='dashed', lw=1, color='grey')
    ax.hlines(0, xmin=ax.xaxis.get_data_interval()[0],
              xmax=ax.xaxis.get_data_interval()[1], linestyles='dashed', lw=1, color='grey')
    #  подписи осей
    ax.set_ylabel('P(default=Yes)')
    ax.set_xlabel('Balance')
    #  легенда
    ax.legend(loc='center left')

### Выбор лучшей модели множественной логистической регрессии   

In [ ]:
# множественная логистическая регрессия
fit_logit_2 = 
fit_logit_2.summary().tables[1]

In [ ]:
# исключаем незначимую объясняющую income
fit_logit_3 = logit(str(''), DF_train).fit(solver='newton-cg')
fit_logit_3.summary().tables[1]

In [ ]:
# пробуем модель со взаимодействием student и balance
fit_logit_4 = logit(str(''), DF_train).fit(solver='newton-cg')
fit_logit_4.summary().tables[1]

In [ ]:
# сводим в таблицу характеристики качества моделей
#  пустые массивы для будущих столбцов
mdls = ["" for x in range(4)]
aics = np.zeros(4)
tprs = np.zeros(4)
spcs = np.zeros(4)

#  цикл по построенным моделям
fits_loop = [fit_logit_1, fit_logit_2, fit_logit_3, fit_logit_4]
for fit in fits_loop :
    #  номер текущей модели в списке
    i = 
    #  объясняющие переменные модели
    mdls[i] = 
    #  значения AIC
    aics[i] = 
    #  делаем прогноз на тестовую
    y_hat_test = 
    y_hat_test = 
    #  значения TPR на тестовой
    tprs[i] = 
    #  значения SPC на тестовой
    spcs[i] = 
    
df_summary = pd.DataFrame({'Объясняющие переменные': mdls, 'AIC': aics,
                          'TPR_test': tprs, 'SPC_test': spcs})
df_summary

Проанализируем таблицу. Наилучшей моделью по информационному критерию Акаике является модель зависимости `defaultYes` от `studentYes` и `balance` (модель №3, наименьшее значение $AIC$ ). Значение чувствительности на тестовой выборке ($TPR_{test}$) у неё также наилучшее. По специфичности ($SPC_{test}$) модель №4 (с переменной константой при `balance`) немного лучше остальных, однако отличие несущественное.  
В итоге стоит остановиться на третьей модели.
Перестроим её с помощью `LogisticRegression()`.   

### Перестраиваем лучшую модель с `LogisticRegression()`  

In [ ]:
# строим модель на обучающей
X_train_LR_2 = DF_train[['studentYes', 'balance']]
fit_LR_2 = LogisticRegression(solver='newton-cg').fit(X=X_train_LR_2, y=y_train)

In [ ]:
# коэффициенты
fit_LR_2.coef_

In [ ]:
# константа
fit_LR_2.intercept_

In [ ]:
# прогноз
X_test_LR_2 = DF_test[['studentYes', 'balance']]
# визуализация матрицы неточностей



In [ ]:
# отчёт по точности на тестовой
y_prob_test_LR_2 = 
y_hat_test = 
print('Модель логистической регрессии от studentYes, balance с порогом 0.5 : \n',
      classification_report(y_test, y_hat_test))

В последнем отчёте метрики качества рассчитаны для каждого класса. Для бинарной классификации:  
* в столбце `recall` для класса 1 стоит чувствительность, а для класса 0 – специфичность.  
* в столбце `precision` для класса 1 стоит ценность положительного прогноза, а для класса 0 – отрицательного.  

---

📚 **Подробнее о прогнозе по модели логистической регрессии**   

$$P(X) = {e^{X \cdot \hat{\beta}} \over (1 + e^{X \cdot \hat{\beta}})}$$  
Здесь $X$ – матрица объясняющих переменных для модели с константой ($n$ строк и $p + 1$ столбцов), $\hat{\beta}$ – вектор-столбец оценок параметров модели ($p + 1$ строк, 1 столбец); $n$ – число наблюдений, $p$ – количество объясняющих переменных модели.  

In [ ]:
# прогноз с помощью logit.predict()
y_hat_test_1 = 

In [ ]:
# снова создаём матрицу объясняющих для модели зависимости defaultYes от balance
X_test = 
# прогноз вручную: y = X * beta, где X – матрица 2000 на 2, а beta – вектор-столбец 2 на 1
y_hat_test = 
# пересчитываем в вероятности
y_hat_test = 

In [ ]:
# совместим результаты прогноза разными методами в одном фрейме
pd.DataFrame({'Прогноз P(X) функцией predict': y_hat_test_1,
              'Прогноз P(X) вручную': y_hat_test.reshape(-1)})

In [ ]:
# перекодируем в 0 и 1, граница отсечения 0.5
y_hat_test = (y_hat_test > 0.5).astype(int)

In [ ]:
# считаем матрицу неточностей
cm = 
# рисуем матрицу в виде тепловой карты
sns.heatmap(cm, annot=True, 
            fmt='g', linewidths=0.5, cmap='BrBG', alpha=0.7)
plt.ylabel('Факт')
plt.xlabel('Прогноз')
plt.show()

---

## ROC-кривая  и подбор порога отсечения 

In [ ]:
# рисуем ROC-кривую


Чтобы выбрать оптимальный порог отсечения формально, получим координаты ROC-кривой с помощью функции `roc_curve()`, а затем свернём их с помощью J-статистики Юдена, которая рассчитывается по формуле:  
$$J = TPR + SPC - 1 = TPR - FPR$$
Когда чувствительность и специфичность модели максимальны, J-статистика также принимает максимальное значение.

In [ ]:
# рассчитываем координаты ROC-кривой
fpr, tpr, thresholds = 

# считаем J-статистику Юдена = TPR - FPR
j_scores = 

df_roc = pd.DataFrame({'TPR': tpr, 'SPC': 1-fpr, 'Порог отсечения': thresholds})
# находим оптимум по максимуму J-статистики


In [ ]:
# сохраняем оптимальный порог
thr = 
# прогноз с новой границей отсечения
y_hat_test = (y_prob_test_LR_2 > thr).astype(int)
# отчёт по точности на тестовой
print('Модель логистической регрессии от studentYes, balance с порогом', np.around(thr, 4), ': \n', 
      classification_report(y_test, y_hat_test))

In [ ]:
# второй способ – порог = доле наименьшего класса на обучающей выборке
thr = 
# прогноз с новой границей отсечения
y_hat_test = (y_prob_test_LR_2 > thr).astype(int)
# отчёт по точности на тестовой
print('Модель логистической регрессии от studentYes, balance с порогом', np.around(thr, 4), ': \n', 
      classification_report(y_test, y_hat_test))

## Линейный дискриминантный анализ (LDA)   

Для начала посмотрим на гистограммы распределения непрерывных объясняющих переменных и проверим их на нормальность.   

In [ ]:
# посмотрим, как работает функция shapiro()


In [ ]:
# статистический тест Шапиро-Уилка на нормальность для объясняющих переменных внутри классов
for col in ['balance', 'income']:
    stat, p = shapiro(DF_train[DF_train.defaultYes == 1][col])
    print(col, '| defaultYes : 1\n', 
          'Statistics=%.2f, p=%.4f' % (stat, p))
    # интерпретация
    alpha = 0.05
    if p > alpha:
        print('Распределение нормально (H0 не отклоняется)\n')
    else:
        print('Распределение не нормально (H0 отклоняется)\n')
    
    stat, p = shapiro(DF_train[DF_train.defaultYes == 0][col])
    print(col, '| defaultYes : 0\n', 
          'Statistics=%.2f, p=%.4f' % (stat, p))
    # интерпретация
    alpha = 0.05
    if p > alpha:
        print('Распределение нормально (H0 не отклоняется)\n')
    else:
        print('Распределение не нормально (H0 отклоняется)\n')

In [ ]:
# тест Лиллиефорса на нормальность
for col in ['balance', 'income']:
    stat, p = lilliefors(DF_train[DF_train.defaultYes == 1][col])
    print(col, '| defaultYes : 1\n', 
          'Statistics=%.2f, p=%.4f' % (stat, p))
    # интерпретация
    alpha = 0.05
    if p > alpha:
        print('Распределение нормально (H0 не отклоняется)\n')
    else:
        print('Распределение не нормально (H0 отклоняется)\n')
        
    stat, p = lilliefors(DF_train[DF_train.defaultYes == 0][col])
    print(col, '| defaultYes : 0\n', 
          'Statistics=%.2f, p=%.4f' % (stat, p))
    # интерпретация
    alpha = 0.05
    if p > alpha:
        print('Распределение нормально (H0 не отклоняется)\n')
    else:
        print('Распределение не нормально (H0 отклоняется)\n')

**Далее построим модели LDA и QDA исключительно в учебных целях.**

In [ ]:
# будем строить модели на непрерывных объясняющих переменных
y_train = 
X_train = 
X_train.head(5)

In [ ]:
# обучаем модель
fit_lda = 

# прогноз на тестовую
X_test = 
y_hat_test = 

In [ ]:
# априорные вероятности классов


In [ ]:
# средние по классам


In [ ]:
# отчёт по точности на тестовой
print('Модель LDA от balance, income: \n', 
     )

## Квадратичный дискриминантный анализ (QDA)

In [ ]:
# обучаем модель
fit_qda = 

# прогноз на тестовую
y_hat_test = 

# отчёт по точности на тестовой
print('Модель QDA от balance, income: \n', 
      )

# Источники 

1. *James G., Witten D., Hastie T. and Tibshirani R.*  An Introduction to Statistical Learning with Applications in R. URL: [http://www-bcf.usc.edu/~gareth/ISL/ISLR%20First%20Printing.pdf](https://drive.google.com/file/d/15PdWDMf9hkfP8mrCzql_cNiX2eckLDRw/view?usp=sharing)     
1. *Jordi Warmenhoven* ISLR-python / github.com. URL: <https://github.com/JWarmenhoven/ISLR-python>  
1. Logistic Regression in Python / realpython.com. URL: <https://realpython.com/logistic-regression-python/>   
1. The k-Nearest Neighbors (kNN) Algorithm in Python / realpython.com. URL: <https://realpython.com/knn-python/> 
1. Руководство по библиотеке Seaborn / из курса "Python для анализа данных" от Физтеха. URL: <https://mipt-stats.gitlab.io/courses/python/09_seaborn.html>  
1. *Tony S. Yu* Matplotlib Style Gallery / tonysyu.github.io. URL: <http://tonysyu.github.io/raw_content/matplotlib-style-gallery/gallery.html>  
1. Intro to data structures / pandas.pydata.org. URL: <https://pandas.pydata.org/pandas-docs/stable/user_guide/dsintro.html>  
1. *Baijayanta Roy* All about Categorical Variable Encoding / towardsdatascience.com. URL: <https://towardsdatascience.com/all-about-categorical-variable-encoding-305f3361fd02>